In [ ]:
%matplotlib inline

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, IntSlider

In [ ]:
def visualize_medical_image(file_path):
    """
    Visualiza una imagen médica almacenada en un archivo .npy con forma [24,2,256,256].
    
    Se despliega una interfaz interactiva para explorar:
      - Time (primer eje, valores de 0 a 23)
      - Modalidad (segundo eje, 0 o 1)
      
    Parámetros:
      file_path: str
          Ruta al archivo .npy (por ejemplo, 'case_01_02.npy')
    """
    # Carga la imagen
    img = np.load(file_path)
    print("Shape de la imagen cargada:", img.shape)
    
    # Función interna que muestra la imagen según los índices seleccionados
    def display_image(slice_index, modality_index):
        plt.figure(figsize=(5, 5))
        plt.imshow(img[slice_index, modality_index, :, :], cmap="gray")
        plt.title(f"Slice {slice_index} - Modalidad {modality_index}")
        plt.axis('off')
        plt.show()
    
    # Crea sliders interactivos para slice y modalidad
    interact(display_image, 
             slice_index=IntSlider(min=0, max=img.shape[0]-1, step=1, 
                                   value=img.shape[0]//2, description="Tiempo"),
             modality_index=IntSlider(min=0, max=img.shape[1]-1, step=1, 
                                      value=0, description="Modalidad"))

# Ejemplo de uso:
file_path = "/data/dev/perfu-net-1/data/train/CTP/case_22_02.npy"  # Ajusta la ruta según corresponda
visualize_medical_image(file_path)


In [ ]:
def generar_histograma_global(ruta_carpeta, indice_c=0, num_bins=100):
    """
    Busca todos los .npy en la carpeta 'ruta_carpeta', asume que cada archivo tiene forma (T, C, H, W).
    Extrae los valores de la dimensión C=indice_c y los acumula en un array global.
    Luego muestra un histograma de todos los valores juntos.
    
    Parámetros:
    -----------
    ruta_carpeta : str
        Ruta a la carpeta que contiene los archivos .npy.
    indice_c : int
        Índice de la dimensión C que se quiere extraer (por defecto 0).
    num_bins : int
        Número de bins en el histograma.
    """
    archivos_npy = sorted(glob(os.path.join(ruta_carpeta, '*.npy')))
    print(f"Encontrados {len(archivos_npy)} archivos en '{ruta_carpeta}'.")
    
    # Lista para acumular los valores de la dimensión c=0 de TODOS los archivos
    all_values = []
    
    for archivo in archivos_npy:
        data = np.load(archivo)  # data.shape debe ser (T, C, H, W)
        
        # Comprobamos que data tenga al menos 2 dimensiones en 'C'
        if data.shape[1] <= indice_c:
            print(f"ADVERTENCIA: El archivo '{archivo}' no tiene C > {indice_c}. Shape: {data.shape}")
            continue
        
        # Extraer la submatriz correspondiente a c=indice_c: data[:, indice_c, :, :]
        data_c0 = data[:, indice_c, :, :]
        
        # Aplanar (flatten) y añadir a la lista
        all_values.append(data_c0.ravel())

        # Obtener min y max
        min_val = data_c0.min()
        max_val = data_c0.max()
        
        # Imprimir
        print(f"Archivo: {os.path.basename(archivo)} | Shape: {data.shape} | Min: {min_val:.2f}, Max: {max_val:.2f}")

    if len(all_values) == 0:
        print("No se pudo extraer ningún dato (verifica las dimensiones de tus archivos).")
        return
    
    all_values = np.concatenate(all_values, axis=0)
    print(f"Tamaño total de datos para histograma: {all_values.size} voxels/píxeles.")
    print(f"Valores mínimos y máximos: {all_values.min()} - {all_values.max()}")
    
    plt.figure(figsize=(6, 4))
    plt.hist(all_values, bins=num_bins, color='blue', alpha=0.7)
    plt.title(f"Histograma global (C={indice_c}) - {len(archivos_npy)} archivos")
    plt.xlabel("Intensidad")
    plt.ylabel("Frecuencia")
    plt.show()

In [ ]:
generar_histograma_global("/data/dev/perfu-net/data/train/CTP")